# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import confusion_matrix
import matplotlib.mlab as mlab
%matplotlib inline
from sklearn import metrics
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from sklearn import tree,metrics

In [3]:
pd.options.display.max_columns=None

# Import Dataset

In [4]:
test = pd.read_csv('test.csv')
test.head()

,ID,Year Type,Year,Organization Group Code,Organization Group,Department Code,Department,Union Code,Union,Job Family Code,Job Family,Job Code,Job,Employee Identifier,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Total Benefits
0,9240,Fiscal,2015,4,Community Health,DPH,DPH Public Health,790,"SEIU - Miscellaneous, Local 1021",2400,"Lab, Pharmacy & Med Techs",2454,Clinical Pharmacist,23148,0.0,0.00,0.00,2203.32,2014.12,4217.44
1,209951,Fiscal,2016,4,Community Health,DPH,DPH Public Health,791,"SEIU - Staff and Per Diem Nurses, Local 1021",2300,Nursing,2328,Nurse Practitioner,8789,0.0,1689.53,31464.76,11872.83,9947.92,53285.51
2,29879,Fiscal,2016,5,Culture & Recreation,LIB,LIB Public Library,790,"SEIU - Miscellaneous, Local 1021",3600,Library,3610,Library Assistant,48290,0.0,932.77,8858.22,9705.96,3764.93,22329.11
3,177914,Fiscal,2015,1,Public Protection,POL,POL Police,21,"Prof & Tech Engineers - Miscellaneous, Local 21",1800,"Budget, Admn & Stats Analysis",1824,Pr Administrative Analyst,43253,0.0,0.00,23955.61,11687.19,8880.83,44523.63
4,180406,Fiscal,2015,4,Community Health,DPH,DPH Public Health,790,"SEIU - Miscellaneous, Local 1021",2900,Human Services,2930,Psychiatric Social Worker,13722,0.0,620.54,20644.61,12384.59,7491.65,40520.85


In [5]:
train = pd.read_csv('train.csv')
train.head()

,ID,Year Type,Year,Organization Group Code,Organization Group,Department Code,Department,Union Code,Union,Job Family Code,Job Family,Job Code,Job,Employee Identifier,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Total Benefits,Class
0,9248,Fiscal,2017,3,Human Welfare & Neighborhood Development,DSS,HSA Human Services Agency,535,"SEIU - Human Services, Local 1021",2900,Human Services,2905,Senior Eligibility Worker,41351,0.00,240.00,11896.36,13765.55,5248.43,30910.34,2
1,44541,Fiscal,2014,6,General Administration & Finance,ASR,ASR Assessor / Recorder,21,"Prof & Tech Engineers - Miscellaneous, Local 21",4200,Appraisal & Taxation,4222,Sr Personal Property Auditor,41792,0.00,400.00,15429.94,9337.37,5599.01,30366.32,2
2,47031,Fiscal,2014,3,Human Welfare & Neighborhood Development,DSS,HSA Human Services Agency,535,"SEIU - Human Services, Local 1021",2900,Human Services,2910,Social Worker,9357,0.00,1080.00,9682.00,8848.03,3463.92,21993.95,2
3,139416,Fiscal,2014,1,Public Protection,FIR,FIR Fire Department,798,"Firefighters - Miscellaneous, Local 798",H000,Fire Services,H002,Firefighter,28022,25730.46,18414.18,24222.26,13911.13,2416.58,40549.97,3
4,123780,Fiscal,2013,2,"Public Works, Transportation & Commerce",MTA,MTA Municipal Transprtn Agncy,790,"SEIU - Miscellaneous, Local 1021",1600,"Payroll, Billing & Accounting",1224,Pr Payroll & Personnel Clerk,51052,1138.28,2148.11,15437.62,12828.15,7246.54,35512.31,3


# Data Preperation

In [6]:
test['Class'] = np.nan

In [7]:
test['group'] = 'test'
train['group'] = 'train'

In [8]:
all = pd.concat((train,test))
all.head()

,ID,Year Type,Year,Organization Group Code,Organization Group,Department Code,Department,Union Code,Union,Job Family Code,Job Family,Job Code,Job,Employee Identifier,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Total Benefits,Class,group
0,9248,Fiscal,2017,3,Human Welfare & Neighborhood Development,DSS,HSA Human Services Agency,535,"SEIU - Human Services, Local 1021",2900,Human Services,2905,Senior Eligibility Worker,41351,0.00,240.00,11896.36,13765.55,5248.43,30910.34,2.0,train
1,44541,Fiscal,2014,6,General Administration & Finance,ASR,ASR Assessor / Recorder,21,"Prof & Tech Engineers - Miscellaneous, Local 21",4200,Appraisal & Taxation,4222,Sr Personal Property Auditor,41792,0.00,400.00,15429.94,9337.37,5599.01,30366.32,2.0,train
2,47031,Fiscal,2014,3,Human Welfare & Neighborhood Development,DSS,HSA Human Services Agency,535,"SEIU - Human Services, Local 1021",2900,Human Services,2910,Social Worker,9357,0.00,1080.00,9682.00,8848.03,3463.92,21993.95,2.0,train
3,139416,Fiscal,2014,1,Public Protection,FIR,FIR Fire Department,798,"Firefighters - Miscellaneous, Local 798",H000,Fire Services,H002,Firefighter,28022,25730.46,18414.18,24222.26,13911.13,2416.58,40549.97,3.0,train
4,123780,Fiscal,2013,2,"Public Works, Transportation & Commerce",MTA,MTA Municipal Transprtn Agncy,790,"SEIU - Miscellaneous, Local 1021",1600,"Payroll, Billing & Accounting",1224,Pr Payroll & Personnel Clerk,51052,1138.28,2148.11,15437.62,12828.15,7246.54,35512.31,3.0,train


In [9]:
all.shape

(212982, 22)

In [10]:
train['Job'].unique().shape

(1146,)

In [11]:
train['Job Code'].unique().shape

(1141,)

In [12]:
train['Job Family Code'].unique().shape

(62,)

In [13]:
train['Job Family'].unique().shape

(57,)

In [14]:
train['Union Code'].unique().shape

(69,)

In [15]:
train['Union'].unique().shape

(77,)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149087 entries, 0 to 149086
Data columns (total 22 columns):
ID                         149087 non-null int64
Year Type                  149087 non-null object
Year                       149087 non-null int64
Organization Group Code    149087 non-null int64
Organization Group         149087 non-null object
Department Code            149087 non-null object
Department                 149087 non-null object
Union Code                 149087 non-null int64
Union                      149087 non-null object
Job Family Code            149087 non-null object
Job Family                 149087 non-null object
Job Code                   149087 non-null object
Job                        149087 non-null object
Employee Identifier        149087 non-null int64
Overtime                   149087 non-null float64
Other Salaries             149087 non-null float64
Retirement                 149087 non-null float64
Health/Dental              149087 non-nul

In [17]:
train.isnull().sum()

ID                         0
Year Type                  0
Year                       0
Organization Group Code    0
Organization Group         0
Department Code            0
Department                 0
Union Code                 0
Union                      0
Job Family Code            0
Job Family                 0
Job Code                   0
Job                        0
Employee Identifier        0
Overtime                   0
Other Salaries             0
Retirement                 0
Health/Dental              0
Other Benefits             0
Total Benefits             0
Class                      0
group                      0
dtype: int64

In [18]:
train['Class'].unique()

array([2, 3, 1], dtype=int64)

In [19]:
train['Year'].unique()

array([2017, 2014, 2013, 2016, 2015], dtype=int64)

In [20]:
all.head(1)

,ID,Year Type,Year,Organization Group Code,Organization Group,Department Code,Department,Union Code,Union,Job Family Code,Job Family,Job Code,Job,Employee Identifier,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Total Benefits,Class,group
0,9248,Fiscal,2017,3,Human Welfare & Neighborhood Development,DSS,HSA Human Services Agency,535,"SEIU - Human Services, Local 1021",2900,Human Services,2905,Senior Eligibility Worker,41351,0.0,240.0,11896.36,13765.55,5248.43,30910.34,2.0,train


In [21]:
train['Organization Group Code'].unique()

array([3, 6, 1, 2, 4, 5, 7], dtype=int64)

In [22]:
col = ['ID','Year','Organization Group Code','Department Code','Union Code',\
       'Job Family Code','Job Code','Overtime','Other Salaries',\
       'Retirement','Health/Dental','Other Benefits','Class','group']

In [23]:
data = all[col]
data.head()

,ID,Year,Organization Group Code,Department Code,Union Code,Job Family Code,Job Code,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Class,group
0,9248,2017,3,DSS,535,2900,2905,0.00,240.00,11896.36,13765.55,5248.43,2.0,train
1,44541,2014,6,ASR,21,4200,4222,0.00,400.00,15429.94,9337.37,5599.01,2.0,train
2,47031,2014,3,DSS,535,2900,2910,0.00,1080.00,9682.00,8848.03,3463.92,2.0,train
3,139416,2014,1,FIR,798,H000,H002,25730.46,18414.18,24222.26,13911.13,2416.58,3.0,train
4,123780,2013,2,MTA,790,1600,1224,1138.28,2148.11,15437.62,12828.15,7246.54,3.0,train


In [24]:
data.shape

(212982, 14)

In [27]:
from sklearn.preprocessing import LabelEncoder

In [28]:
data['Year'] = LabelEncoder().fit_transform(data['Year'])
data['Department Code'] = LabelEncoder().fit_transform(data['Department Code']) 
data['Job Code'] = LabelEncoder().fit_transform(data['Job Code']) 
data['Job Family Code'] = LabelEncoder().fit_transform(data['Job Family Code']) 
data['Union Code'] = LabelEncoder().fit_transform(data['Union Code']) 

C:\Users\Amala\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\Amala\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Amala\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [29]:
data.head()

,ID,Year,Organization Group Code,Department Code,Union Code,Job Family Code,Job Code,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits,Class,group
0,9248,4,3,22,48,18,378,0.00,240.00,11896.36,13765.55,5248.43,2.0,train
1,44541,1,6,5,8,25,515,0.00,400.00,15429.94,9337.37,5599.01,2.0,train
2,47031,1,3,22,48,18,382,0.00,1080.00,9682.00,8848.03,3463.92,2.0,train
3,139416,1,1,28,58,54,1106,25730.46,18414.18,24222.26,13911.13,2416.58,3.0,train
4,123780,0,2,37,54,6,60,1138.28,2148.11,15437.62,12828.15,7246.54,3.0,train


In [30]:
data = data[(data['Other Salaries']>=0)&(data['Overtime']>=0)&(data['Retirement']>=0)&(data['Health/Dental']>=0)&(data['Other Benefits']>=0)]

# Scaling

In [31]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [32]:
data.shape

(212127, 14)

In [33]:
train = data[data['group']=='train']
test = data[data['group']=='test']

In [34]:
train.drop('group',axis=1,inplace=True)

C:\Users\Amala\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [35]:
test.drop(['Class','group'],axis=1,inplace=True)

In [36]:
x = train.drop('Class',axis=1)
y = train['Class']

In [37]:
xtrain,xval,ytrain,yval = train_test_split(x,y,test_size=0.3,random_state=0)

# Decision Tree

In [38]:
clf = tree.DecisionTreeClassifier(max_depth=3)
clf.fit(xtrain,ytrain)  # decision tree for scaled version

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [39]:
val_pred = clf.predict(xval)
val_pred

array([2., 1., 3., ..., 2., 1., 2.])

In [40]:
accuracy_score(val_pred,yval)

0.8923032030706382

# Grid Decision Tree

In [41]:
param_grid = {'criterion':['gini','entropy'],'min_samples_split':[2,10,20],\
              'max_depth':[None,2,5,10],'min_samples_leaf':[1,5,10],'max_leaf_nodes':[None,5,10,20]}

In [42]:
from sklearn.model_selection import GridSearchCV
dt = tree.DecisionTreeClassifier()
clf_g = GridSearchCV(dt,param_grid,cv=2)
clf_g.fit(xtrain,ytrain)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini', 'entropy'], 'min_samples_split': [2, 10, 20], 'max_depth': [None, 2, 5, 10], 'min_samples_leaf': [1, 5, 10], 'max_leaf_nodes': [None, 5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
clf_g.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'max_leaf_nodes': None,
 'min_samples_leaf': 10,
 'min_samples_split': 10}

In [44]:
val_pred_grid = clf.predict(xval)
val_pred_grid

array([2., 1., 3., ..., 2., 1., 2.])

In [45]:
accuracy_score(val_pred_grid,yval)

0.8923032030706382

# Randon forest

In [46]:
param_grid = {'n_estimators':[100],'min_samples_leaf':[1,2,3]}

In [47]:
rf = RandomForestClassifier()
rff = GridSearchCV(rf,param_grid,cv=2)
rff.fit(xtrain,ytrain)

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100], 'min_samples_leaf': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [48]:
rff.best_params_

{'min_samples_leaf': 1, 'n_estimators': 100}

In [49]:
val_pred_rf = rff.predict(xval)
val_pred_rf

array([2., 1., 2., ..., 2., 1., 2.])

In [50]:
accuracy_score(val_pred_rf,yval)

0.9861282575026374

In [51]:
accuracy_score(rff.predict(xtrain),ytrain)

1.0

In [52]:
test_pred_rf = rff.predict(test)
test_pred_rf

array([1., 3., 2., ..., 2., 2., 3.])

In [53]:
test.head()

,ID,Year,Organization Group Code,Department Code,Union Code,Job Family Code,Job Code,Overtime,Other Salaries,Retirement,Health/Dental,Other Benefits
0,9240,2,4,20,54,13,281,0.0,0.00,0.00,2203.32,2014.12
1,209951,3,4,20,55,12,264,0.0,1689.53,31464.76,11872.83,9947.92
2,29879,3,5,35,54,23,483,0.0,932.77,8858.22,9705.96,3764.93
3,177914,2,1,41,8,8,201,0.0,0.00,23955.61,11687.19,8880.83
4,180406,2,4,20,54,18,395,0.0,620.54,20644.61,12384.59,7491.65


In [54]:
df = pd.DataFrame({'ID':test['ID'],'Class':test_pred_rf})

In [55]:
df.head()

,ID,Class
0,9240,1.0
1,209951,3.0
2,29879,2.0
3,177914,3.0
4,180406,3.0


In [56]:
df.to_csv('submission2.csv',index=False)

# BaggingClassifier

In [76]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
model = BaggingClassifier(tree.DecisionTreeClassifier(random_state = 1))
model.fit(xtrain,ytrain)
y_pred = model.predict(xval)
print(accuracy_score(y_pred,yval))

0.9838836389755561


In [ ]:
model.fit(xval,yval)
model.score(xval,yval)

In [77]:
test_pred = model.predict(test)
df = pd.DataFrame({"ID":test['ID'],"Class":test_pred})
df.to_csv("submission6.csv",index = False)

# GradientBoostingClassifier

In [78]:
from sklearn.ensemble import GradientBoostingClassifier
model_4 = GradientBoostingClassifier(random_state=1)
model_4.fit(xtrain,ytrain)
y_pred = model_4.predict(xval)
print(accuracy_score(y_pred,yval))

0.9745684720881687


In [79]:
test_pred = model_4.predict(test)
df = pd.DataFrame({"ID":test['ID'],"Class":test_pred})
df.to_csv("submission7.csv",index = False)